Create an engine from sqlalchemy for the OMOP database

In [36]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [37]:
from sqlalchemy import create_engine
username="admin"
password="test!"
hostname="localhost"
port="5432"
database_name="omop"

connection_string = f"postgresql+psycopg2://{username}:{password}@{hostname}:{port}/{database_name}"
engine = create_engine(connection_string)


Creater a concept finder and a concept mapper

In [38]:
import json
import pandas as pd
from pycarrot import ConceptMapper, ConceptFinder

cfinder = ConceptFinder(engine)
cmapper = ConceptMapper(cfinder) 
cmapper

Load the scan report

In [39]:
df_smr01 = pd.read_excel("./ScanReports/SMR01_ScanReport.xlsx", sheet_name=None)

In [40]:
cmapper.set_scan_report(df_smr01)

Manually map the demographics table

In [41]:
demo = cmapper.map(
    "demographics.csv",
    {"sex": {"1": "8507", "2": "8532"}},
    "encrypted_id",
    "dob",
)
print(json.dumps(demo, indent=6))

{
      "person": {
            "person_id": {
                  "source_table": "demographics.csv",
                  "source_field": "encrypted_id"
            },
            "birth_datetime": {
                  "source_table": "demographics.csv",
                  "source_field": "dob"
            },
            "gender_concept_id": {
                  "source_table": "demographics.csv",
                  "source_field": "sex",
                  "term_mapping": {
                        "1": 8507,
                        "2": 8532
                  }
            },
            "gender_source_value": {
                  "source_table": "demographics.csv",
                  "source_field": "sex"
            }
      }
}


Automatically map the conditions table

In [42]:
tab_1 = cmapper.map(
    "smr01_conditions.csv", "newcondition", "encrypted_id", "admission_date"
)
print(json.dumps(tab_1, indent=6)[:1000]+".....")

{
      "condition_occurrence": {
            "person_id": {
                  "source_table": "smr01_conditions.csv",
                  "source_field": "encrypted_id"
            },
            "condition_start_datetime": {
                  "source_table": "smr01_conditions.csv",
                  "source_field": "admission_date"
            },
            "condition_concept_id": {
                  "source_table": "smr01_conditions.csv",
                  "source_field": "newcondition",
                  "term_mapping": {
                        "A00.0": 4344638,
                        "A00.9": 198677,
                        "A01.0": 192819,
                        "A01.1": 195460,
                        "A01.2": 193953,
                        "A01.3": 442291,
                        "A01.4": 195177,
                        "A02.0": 196328,
                        "A02.1": 40493039,
                        "A02.2": 141209,
                        "A02.8": 133685,
               

Automatically map the conditions table

In [43]:
tab_2 = cmapper.map(
    "smr01_operations.csv", "operation_new", "encrypted_id", "date_operation"
)
print(json.dumps(tab_2, indent=6)[:1000]+".....")

{
      "condition_occurrence": {
            "person_id": {
                  "source_table": "smr01_operations.csv",
                  "source_field": "encrypted_id"
            },
            "condition_start_datetime": {
                  "source_table": "smr01_operations.csv",
                  "source_field": "date_operation"
            },
            "condition_concept_id": {
                  "source_table": "smr01_operations.csv",
                  "source_field": "operation_new",
                  "term_mapping": {
                        "A01.1": 195460,
                        "A01.2": 193953,
                        "A01.3": 442291,
                        "A02.1": 40493039,
                        "A02.2": 141209,
                        "A02.8": 133685,
                        "A02.9": 133685,
                        "A03.1": 4145763,
                        "A03.2": 4185509,
                        "A03.3": 4321384,
                        "A03.8": 440938,
            

Build the rules

In [44]:
rules = {
    **{k: {f"{k}_{i}": v} for i, (k, v) in enumerate(demo.items())},
    **{k: {f"{k}_{i}": v} for i, (k, v) in enumerate(tab_1.items())},
    **{k: {f"{k}_{i}": v} for i, (k, v) in enumerate(tab_2.items())},
}

rules = {"cdm": rules, "metadata": {"dataset": "PHS_SMR01"}}
with open("rules_smr01.json", "w") as f:
    json.dump(rules, f, indent=6)

print(json.dumps(rules, indent=6)[:600]+".....")

{
      "cdm": {
            "person": {
                  "person_0": {
                        "person_id": {
                              "source_table": "demographics.csv",
                              "source_field": "encrypted_id"
                        },
                        "birth_datetime": {
                              "source_table": "demographics.csv",
                              "source_field": "dob"
                        },
                        "gender_concept_id": {
                              "source_table": "demographics.csv",
                              "s.....


In [45]:
demo_alt_f = cmapper.map(
    "demographics.csv",
    {"sex": {"1": "8507"}},
    "encrypted_id",
    "dob",
)
print(json.dumps(demo_alt_f, indent=6))

{
      "person": {
            "person_id": {
                  "source_table": "demographics.csv",
                  "source_field": "encrypted_id"
            },
            "birth_datetime": {
                  "source_table": "demographics.csv",
                  "source_field": "dob"
            },
            "gender_concept_id": {
                  "source_table": "demographics.csv",
                  "source_field": "sex",
                  "term_mapping": {
                        "1": 8507
                  }
            },
            "gender_source_value": {
                  "source_table": "demographics.csv",
                  "source_field": "sex"
            }
      }
}


In [46]:
demo_alt_m = cmapper.map(
    "demographics.csv",
    {"sex": {"2": "8532"}},
    "encrypted_id",
    "dob",
)
print(json.dumps(demo_alt_m, indent=6))

{
      "person": {
            "person_id": {
                  "source_table": "demographics.csv",
                  "source_field": "encrypted_id"
            },
            "birth_datetime": {
                  "source_table": "demographics.csv",
                  "source_field": "dob"
            },
            "gender_concept_id": {
                  "source_table": "demographics.csv",
                  "source_field": "sex",
                  "term_mapping": {
                        "2": 8532
                  }
            },
            "gender_source_value": {
                  "source_table": "demographics.csv",
                  "source_field": "sex"
            }
      }
}


In [47]:
tab_1_alt = cmapper.map(
    "smr01_conditions.csv", "newcondition", "encrypted_id", "admission_date",one_to_one=True
)

tab_2_alt = cmapper.map(
    "smr01_operations.csv", "operation_new", "encrypted_id", "date_operation",one_to_one=True
)


In [48]:
rules_alt = {
    **{"person": {
        "female": demo_alt_f['person'],
        "male": demo_alt_m['person']
    }},
    **{
        k: { 
            f"{k}_{i}": obj
            for i,obj in enumerate(v)
        }
        for k, v in tab_1_alt.items()
    },
        **{
        k: { 
            f"{k}_{i}": obj
            for i,obj in enumerate(v)
        }
        for k, v in tab_2_alt.items()
    }
}

rules_alt = {"cdm": rules_alt, "metadata": {"dataset": "PHS_SMR01"}}
with open("rules_smr01_alternative.json", "w") as f:
    json.dump(rules_alt, f, indent=6)

print(json.dumps(rules_alt, indent=6)[:600]+".....")


{
      "cdm": {
            "person": {
                  "female": {
                        "person_id": {
                              "source_table": "demographics.csv",
                              "source_field": "encrypted_id"
                        },
                        "birth_datetime": {
                              "source_table": "demographics.csv",
                              "source_field": "dob"
                        },
                        "gender_concept_id": {
                              "source_table": "demographics.csv",
                              "sou.....


In [ ]:
df_deaths = pd.read_excel("./ScanReports/Deaths_ScanReport.xlsx", sheet_name=None)
cmapper.set_scan_report(df_deaths)

In [ ]:
demo = cmapper.map(
    "demographics.csv",
    {"sex": {"1": "8507", "2": "8532"}},
    "encrypted_id",
    "dob",
)
print(json.dumps(demo,indent=6))

In [ ]:
tab_1 = cmapper.map("deaths_a.csv", "causedeath", "encrypted_id", "date_of_death")
tab_2 = cmapper.map(
    "deaths_b.csv", {"death": {"1": "4306655"}}, "encrypted_id", "date_of_death"
)

rules = {
    **{k: {f"{k}_{i}": v} for i, (k, v) in enumerate(demo.items())},
    **{k: {f"{k}_{i}": v} for i, (k, v) in enumerate(tab_1.items())},
    **{k: {f"{k}_{i}": v} for i, (k, v) in enumerate(tab_2.items())},
}

rules = {"cdm": rules, "metadata": {"dataset": "PHS_deaths"}}

with open("rules_deaths.json", "w") as f:
    json.dump(rules, f, indent=6)



print(json.dumps(rules, indent=6)[:600]+".....")